<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

# High Dimensional & Deep Learning : Variational Autoencoders

## What is a Variational Autoencoder

A variational autoencoder is an autoencoder that learns a latent variable model for its input data.


What is a variational autoencoder, you ask? It's a type of autoencoder with added constraints on the encoded representations being learned. More precisely, it is an autoencoder that learns a latent variable model for its input data. So instead of letting your neural network learn an arbitrary function, you are learning the parameters of a probability distribution modeling your data. If you sample points from this distribution, you can generate new input data samples: a VAE is a "generative model".

How does a variational autoencoder work?

First, an encoder network turns the input samples x into two parameters in a latent space, which we will note z_mean and z_log_sigma. Then, we randomly sample similar points z from the latent normal distribution that is assumed to generate the data, via z = z_mean + exp(z_log_sigma) * epsilon, where epsilon is a random normal tensor. Finally, a decoder network maps these latent space points back to the original input data.

The parameters of the model are trained via two loss functions: a reconstruction loss forcing the decoded samples to match the initial inputs (just like in our previous autoencoders), and the KL divergence between the learned latent distribution and the prior distribution, acting as a regularization term. You could actually get rid of this latter term entirely, although it does help in learning well-formed latent spaces and reducing overfitting to the training data.



## Objective

* Build a simple Variational Auto Encoder
* Use the Keras Model APi

## Library

In [1]:
from tensorflow.keras.datasets import mnist
import tensorflow.keras.preprocessing.image as kpi
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import tensorflow.keras.losses as kloss
import tensorflow.keras.regularizers as kr
import tensorflow.keras.backend as K
import tensorflow.keras.utils as ku




import numpy as np

import matplotlib.pyplot as plt

## Data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
input_dim = np.prod(x_train.shape[1:])
n_train = x_train.shape[0]
n_test = x_test.shape[0]
x_train = x_train.reshape((n_train, input_dim))
x_test = x_test.reshape((n_test, input_dim))

## Keras Model API

In the previous TP on image classification of the autoencoder notebook we have used the `Sequential` method to build model.

Sequential Method have limits when the architecture we want is not *linear* or when we want to build a layer on top on two different entries.

The `Model` API can sometimes be less intuitive but is much more flexible than the `Sequential` method. We will use it all along this TP.

## A simple Variational Autoencoders

### Param

In [3]:
# network parameters

batch_size=100
intermediate_dim = 512
latent_dim = 2
epochs = 50

### Encoder

#### Latent Variable

TODO add image here. <br>
TODO explain why log_var. <br>
We first build the encoder part. <br>
It is first composed of  a `Dense` layer with 512 neurons. <br>
Two `Dense` layer are then added on **top of the same layer**. These two layers will produce the two variable *z_mean* and  *z_log_var* in the latent space.

Note that we define each layer as a function of an input which is the output of the layer he follows.

In [4]:
# build encoder model
inputs = kl.Input(shape=(784,), name='encoder_input')
x = kl.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = kl.Dense(latent_dim, name='z_mean')(x)
z_log_var = kl.Dense(latent_dim, name='z_log_var')(x)

Instructions for updating:
Colocations handled automatically by placer.


#### The stochastic latent variable

We use the reparametrization trick to define a random variable z that is conditioned on the input image x as follows:

$$ z \sim \mathcal{N}(\mu_z(x), \sigma_z(x)) $$

<img src="image/vae_3.svg" width="600px" />



The reparametrization tricks defines $z$ has follows:

$$ z = \mu_z(x) + \sigma_z(x) \cdot \epsilon$$

with:

$$ \epsilon \sim \mathcal{N}(0, 1) $$

This way the dependency to between $z$ and $x$ is deterministic and differentiable. The randomness of $z$ only stems from $\epsilon$ only for a given $x$.

Note that in practice the output of the encoder network parameterizes $log(\sigma^2_z(x)$ instead of $\sigma_z(x)$. Taking the exponential of $log(\sigma^2_z(x)$ ensures the positivity of the standard deviation from the raw output of the network:

In [5]:
# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim))
    return z_mean + K.exp(z_log_sigma) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
z = kl.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

So far we have defined all the layer we need to build the encoder part of the vae. <br> 
We now define the model, with the `Model`method. For that we just have to specify the input layer and the output we want (here, it's the latent variable z). The model is then build automatically.

TODO allow plot model

In [6]:
# instantiate encoder model

encoder = km.Model(inputs, z, name='encoder')
encoder.summary()
#ku.plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 784)          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          401920      encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 2)            1026        dense[0][0]                      
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 2)            1026        dense[0][0]                      
__________________________________________________________________________________________________
z (Lambda)

### Decoder 

The decoder take as an input the a vector z (which is a sample of the latent distribution define by the encoder). 
it is then compose of 2 Dense layers with the following caracteristics : 

* 512 neurons, relu activation
* 784 neurons (input_shape), sigmoid activation

**Exercise** build this simple model using the Model API

In [12]:
# %load solutions/decoder_vae.py
# build decoder model
latent_inputs = kl.Input(shape=(latent_dim,), name='z_sampling')
x = kl.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = kl.Dense(784, activation='sigmoid')(x)

# instantiate decoder model
decoder = km.Model(latent_inputs, outputs, name='decoder')

In [13]:
decoder.summary()
#plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True) TODO


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               1536      
_________________________________________________________________
dense_3 (Dense)              (None, 784)               402192    
Total params: 403,728
Trainable params: 403,728
Non-trainable params: 0
_________________________________________________________________


### Vae

We now associate the two model to define our Variational Autoencoder

In [14]:
# instantiate VAE model
outputs = decoder(encoder(inputs))
vae = km.Model(inputs, outputs, name='vae_mlp')
vae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 784)               0         
_________________________________________________________________
encoder (Model)              (100, 2)                  403972    
_________________________________________________________________
decoder (Model)              multiple                  403728    
Total params: 807,700
Trainable params: 807,700
Non-trainable params: 0
_________________________________________________________________


In [17]:

reconstruction_loss = kloss.binary_crossentropy(inputs,
                                              outputs)

reconstruction_loss *= 784
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 784)               0         
_________________________________________________________________
encoder (Model)              (100, 2)                  403972    
_________________________________________________________________
decoder (Model)              multiple                  403728    
Total params: 807,700
Trainable params: 807,700
Non-trainable params: 0
_________________________________________________________________


In [ ]:
vae.fit(x_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test, None))
vae.save_weights('vae_mlp_mnist.h5')

Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
60000/60000 [==============================] - 7s 116us/sample - loss: 189.7898 - val_loss: 168.9534
Epoch 2/50
60000/60000 [==============================] - 8s 134us/sample - loss: 165.5808 - val_loss: 163.2383
Epoch 3/50
60000/60000 [==============================] - 9s 143us/sample - loss: 161.7854 - val_loss: 160.7026
Epoch 4/50
60000/60000 [==============================] - 10s 169us/sample - loss: 159.5383 - val_loss: 158.9314
Epoch 5/50
60000/60000 [==============================] - 10s 174us/sample - loss: 157.6584 - val_loss: 156.9970
Epoch 6/50
60000/60000 [==============================] - 11s 179us/sample - loss: 156.0020 - val_loss: 155.7725
Epoch 7/50
60000/60000 [==============================] - 10s 174us/sample - loss: 154.5752 - val_loss: 154.4151
Epoch 8/50
60000/60000 [==============================] - 12s 200us/sample - loss: 153.4234 - val_loss: 153.6388
E

## Classification of latent variable

In [ ]:
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test, cmap="Set1")
plt.colorbar()
plt.show()

# Generation of number

In [ ]:
# display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# we will sample n points within [-15, 15] standard deviations
grid_x = np.linspace(-15, 15, n)
grid_y = np.linspace(-15, 15, n)

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.show()